In [2]:
!pip install torch torchvision torchaudio

  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached numpy-1.23.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached certifi-2022.9.24-py3-none-any.whl (161 kB)


You should consider upgrading via the 'C:\Users\Aaditya\PycharmProjects\pythonProject6\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
!pip install stable-baselines3[extra] protobuf==3.20.*

In [60]:
!pip list

Package                    Version
-------------------------- -----------
absl-py                    1.3.0
ale-py                     0.7.5
anyio                      3.6.2
argon2-cffi                21.3.0
argon2-cffi-bindings       21.2.0
asttokens                  2.1.0
attrs                      22.1.0
AutoROM                    0.4.2
AutoROM.accept-rom-license 0.4.2
backcall                   0.2.0
beautifulsoup4             4.11.1
bleach                     5.0.1
cachetools                 5.2.0
certifi                    2022.9.24
cffi                       1.15.1
charset-normalizer         2.1.1
click                      8.1.3
cloudpickle                2.2.0
colorama                   0.4.6
contourpy                  1.0.6
cycler                     0.11.0
debugpy                    1.6.3
decorator                  5.1.1
defusedxml                 0.7.1
entrypoints                0.4
executing                  1.2.0
fastjsonschema             2.16.2
fonttools                 

You should consider upgrading via the 'C:\Users\Aaditya\PycharmProjects\pythonProject6\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install pydirectinput mss

You should consider upgrading via the 'C:\Users\Aaditya\PycharmProjects\pythonProject6\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install pytesseract

You should consider upgrading via the 'C:\Users\Aaditya\PycharmProjects\pythonProject6\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from mss import mss
import numpy as np
from matplotlib import pyplot as plt
import time
import cv2
import pydirectinput
import pytesseract
pytesseract.pytesseract.tesseract_cmd = \
r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from gym import Env
from gym.spaces import Box, Discrete

In [2]:
class WebGame(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        self.action_space = Discrete(3)
        self.cap = mss()
        self.game_location = {'top':300, 'left':0, 'width':600, 'height':500}
        self.done_location = {'top':405, 'left':630, 'width':660, 'height':70}


    def step(self, action):
        # Action key : 0 = Space, 1 = Duck, 2 = No Action
        action_map = {
            0:'space',
            1:'down',
            2:'no_op'
        }
        if action != 2:
            pydirectinput.press(action_map[action])
        done, done_cap = self.get_done()
        new_observation = self.get_observation()
        reward = 1
        info = {}
        return new_observation, reward, done, info
    def render(self):
        cv2.imshow('Game', np.array(self.cap.grab(self.game_location))[:,:,:3])
        if cv2.waitKey(1) & 0xFF == ord('q'):
            self.close()
    def reset(self):
        time.sleep(1)
        pydirectinput.click(x=150, y=150)
        pydirectinput.press('space')
        return self.get_observation()
    def close(self):
        cv2.destroyAllWindows()
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3]
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (100,83))
        channel = np.reshape(resized, (1,83,100))
        return channel
    def get_done(self):
        done_cap = np.array(self.cap.grab(self.done_location))[:,:,:3]
        done_strings = ['GAME','GAHE']
        done = False
        res = pytesseract.image_to_string(done_cap)[:4]
        if res in done_strings:
            done = True
        return done, done_cap

In [3]:
env = WebGame()

In [63]:
for episode in range(3):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        obs, reward, done, info = env.step(env.action_space.sample())
        total_reward += reward
    print(f'Total reward episode {episode} is {total_reward}')

Total reward episode 0 is 11
Total reward episode 1 is 9
Total reward episode 2 is 11


In [4]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

In [5]:
env_checker.check_env(env)

FailSafeException: PyDirectInput fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pydirectinput.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [11]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq ==0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True


In [12]:
CHECKPOINT_DIR = './train/'
LOG_DIR='./logs/'

In [13]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)


In [14]:
from stable_baselines3 import DQN


In [15]:
model = DQN('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=600000, learning_starts=200)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [33]:
model.learn(total_timesteps=500, callback=callback)


Logging to ./logs/DQN_4
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 9.5      |
|    ep_rew_mean      | 9.5      |
|    exploration_rate | 0.278    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1        |
|    time_elapsed     | 29       |
|    total_timesteps  | 38       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 9.38     |
|    ep_rew_mean      | 9.38     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1        |
|    time_elapsed     | 56       |
|    total_timesteps  | 75       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 9.42     |
|    ep_rew_mean      | 9.42     |
|    exploration_rate | 0.05     |
| time/               |        

KeyboardInterrupt: 

 EST MODEL

In [19]:
model.load(os.path.join('train/best_model_88000'))

In [20]:
model

In [17]:
for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        total_reward += reward
    print(f'Total reward episode {episode} is {total_reward}')


Total reward episode 0 is 14
Total reward episode 1 is 8


KeyboardInterrupt: 